#how it works:\
0-find messages by keywords\
  drop fundraise of big funds/bloggers\
1-parse all numbers in text + money sign at the beginning if exists (\\$,₴,euro)\
2-remove part after the dot (1234.00 -> 1234)\
3-calculate "money" for each number found using multipliers (\\$ -> 40, ₴ -> 1, etc)\
4-take the biggest one


#TODO:\
(p0) - understand sample size\
(p0) - handle dots and commas properly \
(p1) - handle intermediate posts ("лишилось",..)\
(p3) - handle multiple fundrasings in 1 message (how???)\
        now just maximum number is returned\
(p1) - for others channes add some logic to handle fundraise (i.e. start, intermediate calls, end)\
            probably can just drop the same numbers following a week after 1st message (usually amount of money is different for each call)\
(p2) - try other channels\
Орієнтовна вартість 7-9 тис.$.

In [98]:
import re

#TODO:
#dorp ZIPCODES
#ми зібрали близько 40-ка мільйонів гривень
def preprocess_msg(msg):
    test_str = msg.lower()
    #drop news about big fonds, president, government and messeges without digits
    if not re.search(r'\d', test_str):
        return ""
    if re.search(r'Притул|Лачен|Лаченков|Повернись Живим|Чмут|UNITED24|Міноборон|звернення Президента|Зеленськ|СБУ|байрактар|Дайджест', 
                 test_str, flags=re.IGNORECASE):
        return ""
    #remove spaces between numbers (for 750 000$ and card number formats)
    test_str = re.sub('(\d)\s(\d)', r'\1\2', test_str) 
    #drop parentheses
    test_str = re.sub('\(|\)', '', test_str) 
    #drop phone number
    test_str = re.sub('номер телефону.*', '', test_str) 
    test_str = re.sub('\s0\d+', '', test_str) 
    test_str = re.sub('\s380\d+', '', test_str) 
    #handle edge cases
    test_str = re.sub('\xa0', ' ', test_str) 
    #print(test_str)
    test_str = re.sub('\(http.*\)|http.*[\s\n]', '', test_str)
    #remove КОД ЄДРПОУ
    test_str = re.sub('код єдрпоу[ :\n`]+[0-9]+|код[ :\n`]+[0-9]+|єдрпоу[ :\n`]+[0-9]+', '', test_str) 
    return test_str

In [93]:
def find_money(test_str):
    return re.findall('[$₴€]?\d+[,\.\d+]?[,\.\d]+[\s+\.]?[\w$₴€]+', test_str)

#print(df_all_unique.iloc[126].date)
#find_money(df_all_unique.iloc[110].message_text)
#find_money('ми зібрали близько 1,34 мільйонів гривень')


In [48]:
def clean_money_list(money_list):
    cleaned_list = []
    for list_item in money_list:
        list_item = re.sub('\.([0-9][0-9][0-9])', r'\1', list_item)
        list_item = re.sub('\.[0-9][0-9]\s', '', list_item)
        
        #1,23 млн -> 1млн..
        list_item = re.sub('\,[0-9][0-9]\s', '', list_item)

        list_item = re.sub('\s', '', list_item)
        cleaned_list_item = list(filter(None, re.split('([,\d]+)', list_item)))
        cleaned_list.append(cleaned_list_item)
    return cleaned_list


#text = 'Тиждень тому ми закликали ще раз підтримати збір на підрозділ Харківської Нацгвардії і за тиждень зібрали 180.000 гривень їм на круті легкі британські бронежилети, а разом вже є 340.000 гривень з 500.00 запланованих. Лишилось зібрати якісь 160 тисяч на ще декілька тактичних шоломів і 1,23 млн, тепловізори і усякі окуляри-навушники.'
#money_list = find_money(text)
#cleaned_money_list = clean_money_list(money_list)  
#donation = parse_money_list(cleaned_money_list)
#cleaned_money_list

[['180000', 'гривень'],
 ['340000', 'гривень'],
 ['500', 'запланованих'],
 ['160', 'тисяч'],
 ['1', 'млн']]

In [118]:
def parse_money_list(test_list):   
    money_list = [0]
    for list_item in test_list:
        list_len = len(list_item)
        #print(list_len)
        if (list_len > 2):
            #print(f'invalid list: {list_item}')
            continue
        #if only digit has been found and it is greater then 2500 (not year, month, hour, some amount of equipement, etc)
        #drop card numbers, IPN, IBAN
        #we assume it is UAH donation
        elif (list_len == 1):
            if (list_item[0].isdigit() and len(list_item[0]) < 10 and int(list_item[0]) > 2500):
      #          print(f'append with {list_item[0]}')
                money_list.append(int(list_item[0]))
            else:
                continue
  
        uah_donation = 0
        multiplier = 1
        #print(list_item)
        for list_str in list_item:
            if (list_str.isdigit()):
                #skip cards, IPN,..
                if (len(list_str) > 7):
                    uah_donation = 0
                else:
                    uah_donation = int(list_str)
                continue

            match list_str:
                case "тис":
                    multiplier = 1000
                case "тисяч":
                    multiplier = 1000
                case "тысяч":
                    multiplier = 1000
                case "тыс":
                    multiplier = 1000
                case "тисячі":
                    multiplier = 1000
                case "к":
                    multiplier = 1000
                case "млн":
                    multiplier = 1000000
                case "мільйонів":
                    multiplier = 1000000
                case "мільйони":
                    multiplier = 1000000
                case "мільйона":
                    multiplier = 1000000
                case "$":
                    multiplier = 40
                case "доларів":
                    multiplier = 40
                case "долларов":
                    multiplier = 40
                case "долар":
                    multiplier = 40
                case "грн":
                    multiplier = 1
                case "гривень":
                    multiplier = 1
                case "₴":
                    multiplier = 1
                case "€":
                    multiplier = 40
                case "євро":
                    multiplier = 40
                case other:
                    #assume if 1st digit > 10k, but less then 1 million then it is money in uah
                    if (uah_donation > 10000 and uah_donation < 1000000):
                        multiplier = 1
                    else:
                        multiplier = 0
                        
        #handle: '19 000 тисяч; 190.000 тисяч'
        if (multiplier == 1000 and uah_donation > 10000):
            multiplier = 1
                
        money_list.append(uah_donation * multiplier)
        #print(f'for list {list_item} uah: {uah_donation}, multiplier {multiplier}')
            
    return max(money_list)

In [96]:
def find_product(test_str):
    text = test_str[::]
    text = text.lower()
    dic_numbers = {" один ":" 1 ", " два ":" 2 ", " другий ":" 2 ", " три ":" 3 ", " трьох ": " 3 ", " чотири ":" 4 ", " п'ять ":" 5 ", " шість ":" 6 ", " сім ":" 7 ", " вісім ":" 8 ", " дев'ять ":" 9 ", " десять ":" 10 "}
    for i, j in dic_numbers.items():
        text = text.replace(i, j)
    delimiters = "\n", ", ", ". ", " та ", " і "
    regex_pattern = '|'.join(map(re.escape, delimiters))
    sent_text = re.split(regex_pattern, text)
    sent_text = [value for value in sent_text if value != '']
    key_products = {'мінна безпека': ['металошукач', 'розтяжок', 'лопат'],
                  'БПЛА': ['бпла', 'бпак'],
                  "комп'ютерна та відео техніка": ['ноутбук', 'планшет', 'айпад', '3d сканер', 'годинник', 'gopro', 'фотопастк'],
                  'тепловізори, снайпінг, ПНБ': ['колліматор', 'коліматор', 'тепловізор', 'теплик', 'теплак', 'прилади нічного бачення', 'нічники', ' пнб',
                                                 'глушник', 'глушителі'],
                  'авто': ['автівок', 'автівк', 'бусік', 'позашляховик', 'швидка', ' бус ', ' бус. ', 'пікап', 'мікроавтобус',
                           'вантажівка', 'машина', 'санмашина', 'трал', 'транспорт'],
                  "зв'язок": ['радіостанці', 'рації', 'рація', 'комплектів рацій', 'радєйк', 'ретранслятор', 'радійк', 'radio', 'телефон', 'смартфон',
                              'антена', 'автоантена', 'двохдиапазонних антен', 'трансивер', 'starlink', 'старлінк'],
                  'запчастини': ['комплекти зимової гуми', 'мастил', 'комплекти коліс', 'комплекти шин', 'пальне', 'комплектуючі', 'аксесуар', 'кріплення',
                                 'коробів для пкм', 'маскувальних сіток', 'маскувальна сітка', 'буржуйк', 'обігрівач', 'буржуйок'],
                  'квадрокоптери': ['dji matrice', 'пташк', 'квадракоптер', 'дрон', 'mavic', 'коптер'],
                  'броня': ['комплектів плит', 'бронік', 'бронезахист', 'бронежилет'],
                  'оптика': ['приціл', 'збільшувач', 'прицел', 'термоприціл', 'монокуляр', 'біноклі', 'дальномір', ],
                  'засоби спостереження': ['метеостанція', 'відеоспостереження'],
                  'медицина': ['турникет', 'джгут', 'тактичних аптечок', 'тактичні аптечки', 'аптечок', 'аптечк', 'пульсоксиметр', 'тонометр', 
                               "пістолет для ін'єкцій", "м'яких нош"],
                  'генератори і акумулятори': ['генератор', 'зарядних пристроїв', 'батаре', 'акумулятор', 'зарядки', 'зарядні пристрої', 'пауербанк', 'повербанк',
                                               'сонячні панелі'],
                  'ремонт': ['гідратор', 'бензопила', 'домкрат', 'інструмент', 'електроінструмент', 'стволи для гасіння', 'обладнання', 'екскаватор', 'спецобладнання'],
                  'амуніція': ['форма', 'форму', 'амуніці', 'екіпіруванн', 'шкарпетки', 'наушник', 'навушник', 'окуляр',
                               'перчатки', 'панами', 'футболки', 'сорочки', 'тактичних рюкзаків', 'кепка', 'маск-халат', 'маскувальний костюм', 
                               'маскувальні костюми', 'комбінізон', 'плітоноск', 'ременно-плечова система', 'засоби індивідуального захисту', ' рпс',
                               'наколінник', 'рукавиц', 'перчів', 'панам', 'розгрузк', 'балаклав', 'спорядження',
                               'термобілизна', 'флісок', 'дощовик', 'ліхтар', 'ліхтарик', 'мультитул', ' нож', ' ніж', 'одноразові душі', 'каремати', 'спальник',
                               'касок', 'каски', 'шолом', 'шлем', 'пари тактичних кросівок', 'пар взуття', 'берці', 'пар кросівок', 'пар кросівок',
                               'пар тактичного взутя', 'чобот', 'рюкзак', 'подсумки', 'сумки', 'підсумкам', 'наплічник'],
                  'зброя': ['гранатомет', 'оружие','зброя', 'боекомплект', 'протидронова рушниця', 'антидронових пристрої', 'глушилка', 'антидронов'],
                  'гуманітарна допомога': ['медикаменты', 'медикаменти', 'інсулін', 'ліки', 'гуманитарк', 'гуманітарн', 'харчуванн', 'їжа'],
                  }
    product_count = dict()
    for sentence in sent_text:
        prod_num_in_sent = 0
        for key, prod_category in key_products.items():
            for prod in prod_category:
                if prod in sentence:
                    prod_num_in_sent += 1
                    if prod_num_in_sent > 1:
                        continue
                    pattern = f"\d+ шт|\d+ {prod}"
                    found_data = re.search(pattern, sentence)
                    if found_data:
                        amount = re.search(r"\d+", found_data[0])
                        if key not in product_count.keys():
                            product_count[key] = int(amount[0])
                        else:
                            product_count[key] += int(amount[0])
                    else:
                        if key not in product_count.keys():
                            product_count[key] = 1
                        else:
                            product_count[key] += 1

    return product_count



In [5]:
#Installing theleton
from telethon import TelegramClient, functions, types
from asyncio import run


API_ID=XXXXXXXX
API_HASH=''
SESSION_FILE='session1'

In [6]:
from telethon.tl.types import PeerChannel
from datetime import datetime, timezone

async def get_channel_entity(search_str, api_id, api_hash):
    async with TelegramClient(SESSION_FILE, api_id, api_hash) as client:
        channel = await client.get_entity(search_str)
        return channel

In [119]:
war_started = datetime(2022, 2, 24, 1, 30, 0, 0, timezone.utc)
async def search_messages(search_str, entity, city, api_id, api_hash):
    async with TelegramClient(SESSION_FILE, api_id, api_hash) as client:    
        rows_list = []
        #TODO: is there a way to set limit "from-to"?
        async for message in client.iter_messages(entity=entity, search=search_str, limit=100):
            if (message.date < war_started):
                break;
            
            msg = preprocess_msg(message.text)
            money_list = find_money(msg)
            money_str = '\n'.join(money_list)
            cleaned_money_list = clean_money_list(money_list)  
            donation = parse_money_list(cleaned_money_list)
            products = find_product(message.text)
            
            message_dict = {}
            message_dict.update({
                               "channel_id" : entity.id, 
                               "message_id" : message.id, 
                               "date" : message.date,
                               "channel_name" : entity.title,
                               "city" : city,
                               "message_text" : message.text,
                               "views" : message.views,
                               "forwards" : message.forwards,
                               "tmp_keyword" : search_str,
                               "donate" : donation,
                               "tmp_possible_donates" : money_str,})
            message_dict.update(products)
            rows_list.append(message_dict) 
        return rows_list

In [120]:
#%%time

import pandas as pd

# df_all = pd.DataFrame(columns=['channel_id', 'message_id', 'date', 'channel_name', 'city', 'message_text',\
#                              'views', 'forwards', 'tmp_keyword', 'donate', 'tmp_possible_donates', 'forwarded_from_id', 'forwarded_from_name'])
df_all = pd.DataFrame()

search_filter = ['збір коштів', 'збираємо гроші', 'оголошуємо збір', 'плануємо закупити', 'починаємо збір', 'новий збір'
                     'paypal', 'monobank', 'реквізити приват', 'новий збір', 'сума збору', 'збір ціль'
                     'сумма сбора', 'монобанка', 'банка сбора', 'сбор мы потратили', 'сбор сумма',
                     'збір закрито', 'закрили збір', 'звіт закупили', 'звіт купили', 'звіт передали', 'звіт придбали', 
                     'звітуємо по коштам', 'задонатили', 'ви зібрали', 'ми зібрали']

search_channels = [
    {"Kharkiv" : ["hueviykharkov", "kharkiv_1654", "kharkivlife"]},
    {"Lviv" : ["privatnamemarnya", "lvivych_news", "varta1"]},
    {"Poltava": ["suspilnepoltava", "my_gotovi_poltava"]},
    {"Dnipropetrovsk" : ["hyevuy_dnepr", "dnepr_operativ", "dnipro_now"]},
    {"Sumy": ["suspilnesumy", "sumygo", "sumyliketg"]},
    {"Zaporizhzhia" : ["eto_zp", "info_zp", "etozp"]},
    {"Kherson" : ["hueviyherson", "nadzor_kherson"]},
    {"Mykolaiv" : ["vanek_nikolaev", "truexanikolaev", "h_mykolaiv2022"]},
    {"Kyiv" : ["prankota_ch", "kievreal1", "kyivoperativ", "VolunteerTalksKyiv", "UKRAINEALIVE22", "ironvolunteers"]},
    {"Ukraine" : ["truexanewsua", "UkraineNow", "uniannet", "operativnoZSU", "a_shtirlitz", "informnapalm", "KARAS_EVGEN", "yigal_levin", "valerii_markus", "kapelan_it", "gistapapapa", "The3rdForceUA", "dvish_alive", "ukrarmed_forces", "bochkala_war", "istoriya_ukrainy", "OstanniyCapitalist", "ssternenko", "oko_original", "zloyodessit"]}
    ]

for city in search_channels:
    city_name, channels = list(city.items())[0]
    print(city_name)
    for channel in channels:
        print(channel)
        channel_entity = await get_channel_entity(channel, API_ID, API_HASH)
        for filter_str in search_filter:
            data = pd.DataFrame(await search_messages(filter_str, channel_entity, city_name, API_ID, API_HASH))
            if df_all.shape[0] == 0:
                df_all = data.copy()
            df_all = pd.concat([df_all, data], ignore_index=True)

#TODO: handle middle results "залишилось зібрати" "вже зібрали"
        

Kharkiv
hueviykharkov
kharkiv_1654
kharkivlife
Lviv
privatnamemarnya
lvivych_news
varta1
Poltava
suspilnepoltava
my_gotovi_poltava
Dnipropetrovsk
hyevuy_dnepr
dnepr_operativ
dnipro_now
Sumy
suspilnesumy
sumygo
sumyliketg
Zaporizhzhia
eto_zp
info_zp
etozp
Kherson
hueviyherson
nadzor_kherson
Mykolaiv
vanek_nikolaev
truexanikolaev
h_mykolaiv2022
Kyiv
prankota_ch
kievreal1
kyivoperativ


In [138]:
pd.set_option('display.max_colwidth', None)

df_all_unique = df_all.drop_duplicates(subset='message_id', keep="first", ignore_index=True)
df_all_unique = df_all.drop_duplicates(subset='tmp_possible_donates', keep="first", ignore_index=True)
df_all_unique.date = df_all_unique.date.dt.tz_localize(None)
df_all_unique.to_excel('donates_all.xlsx', index=False)
df_all_unique[(df_all_unique['donate'] < 10000000) & (df_all_unique['donate'] > 1000)].to_excel('donates_filtered.xlsx', index=False)

/var/folders/ds/npzd8ntn57vb2bqfy98srpbh0000gn/T/ipykernel_15123/1218144255.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_all_unique.date = df_all_unique.date.dt.tz_localize(None)


In [89]:
for city in search_channels:
    city_name, channels = list(city.items())[0]
    city_df = df_all_unique[df_all_unique['city'] == city_name]
    #drop general news about donations"
    donations = city_df[city_df['donate'] < 10000000].donate.sum() / 1000000.0
    views = city_df[(city_df['donate'] < 10000000) & (city_df['donate'] > 1000)].views.sum()
    print(f'\nfor city {city_name} found {city_df.shape[0]} fundraising messages and raised {donations} million uah')
    print(f'total views of the donation posts: {views/1000000.0} million, donations/viewes: {donations/views}')


for city Kharkiv found 292 fundraising messages and raised 73.62305 million uah
total views of the donation posts: 29.448329 million, donations/viewes: 2.5000756409642125e-06

for city Lviv found 462 fundraising messages and raised 130.338433 million uah
total views of the donation posts: 10.298651 million, donations/viewes: 1.2655874347038268e-05

for city Poltava found 144 fundraising messages and raised 13.912819 million uah
total views of the donation posts: 1.041848 million, donations/viewes: 1.3353981578886749e-05

for city Dnipropetrovsk found 158 fundraising messages and raised 25.889553 million uah
total views of the donation posts: 12.908542 million, donations/viewes: 2.00561403448972e-06

for city Sumy found 122 fundraising messages and raised 22.730948 million uah
total views of the donation posts: 1.437285 million, donations/viewes: 1.58151987949502e-05

for city Zaporizhzhia found 62 fundraising messages and raised 3.779102 million uah
total views of the donation posts: 

In [127]:
city_df[(city_df['donate'] < 10000000) & (city_df['donate'] > 1000)]

,channel_id,message_id,date,channel_name,city,message_text,views,forwards,tmp_keyword,donate,tmp_possible_donates,forwarded_from_id,forwarded_from_name
0,1149657308,91699,2022-12-11 14:02:13,Хуевый ✙ Харьков 🇺🇦,Kharkiv,"Великий збір NATA& NO PANIC FOUNDATION на 3 пікапа та 3 MAVIC 3 ще триває.\n\n👉знято з банки 251000грн, з інших карт 74000грн., загальна сума банки змінена. \n1.Вже закрили збір на одне авто та купили L200 для 58ї\n2.На авто для 211спецбатальону не вистачає 1000$\n3.І ще третя авто Кракену + 3 MAVIC \n\nЗалишилося зібрати 425000грн\n\n🔗Посилання на Банку\nhttps://send.monobank.ua/jar/9n75DzdAkM\n\n💳Номер картки Банки\n5375 4112 0181 6381\n\n👉ПРИВАТ\n5168745032031723\n\n👉ВАЛЮТНА\n4149499160468763\n\n👉 МОНО\n5375411417173569\n\n👉 PayPal nikitosnata1982@gmail.com\n\nСЛАВА ЗСУ 💪\nПИЗДА ₚУСЬНІ 😈",137394,1,збір коштів,425000,251000грн\n74000грн\n1.Вже\n200 для\n58ї\n2.На\n211спецбатальону\n1000$\n3.І\n425000грн\n5375411201816381\n5168745032031723\n4149499160468763\n5375411417173569\n1982,NaN,NaN
1,1149657308,90846,2022-12-05 09:00:57,Хуевый ✙ Харьков 🇺🇦,Kharkiv,"Друзі привіт,\nБроварня TEN MEN розпочинає другий збір коштів для наших воїнів.\n\nБійцям «Сил Спеціальних Операції» (ССО) з Харкова потрібні гарні ноутбуки, щоб нищити русню.\n\nЦі ноутбуки будуть використані для дуже, дуже гарних справ.\nЄ потреба у трьох штуках.\n\n🎯 Ціль: 124 000 ₴\n\n🔗Посилання на банку\nhttps://send.monobank.ua/jar/3JTbxUmMEb\n\n💳Номер картки банки\n5375 4112 0460 0618",139460,36,збір коштів,124000,124000 ₴\n5375411204600618,NaN,NaN
2,1149657308,90178,2022-11-29 12:45:31,Хуевый ✙ Харьков 🇺🇦,Kharkiv,"Пробили 200к на другий пікап для HMV\n\nПотрібно добити ще 150 тисяч, давайте швиденько закриємо й цей збір\n\nПосилання на банку:\nhttps://send.monobank.ua/jar/NwjiMsQA8",158183,0,збір коштів,200000,200к\n150 тисяч,NaN,NaN
3,1149657308,87903,2022-11-12 09:00:06,Хуевый ✙ Харьков 🇺🇦,Kharkiv,"Вірю в ЗСУ та силу ХХ🤗\n\n🫡 NATA& NO PANIC FOUNDATION \nВідкриває новий важливий збір!\n\n🥳Дякуємо нашим ЗСУ за звільнення Херсона!\nАле на Бахмутському та Донецькому напрямку зараз дуже важко, хлопцям потрібна наша допомога.\nБез тачки неможливо виконувати бойові завдання, вирішувати логістичні проблеми чи евакуювати пораненних. \n\n🛻 Бійцям 58 бригади потрібен пікап або позашляховик, щоб давати рузькім пизди на Бахмутському напрямку. \n\n🛻 🛻 Два пікапа потрібно 92 бригади та Кракену на Донецькому напрямку. Тут без коментарів, ви і так знаєте ,як вони їбашать і чому так потрібні авто.\n\n🔥 ТРИ MAVIC 3 та додаткові батареї до них. Хлопцям для розвідки та скидання ВОГів на голови орків.\n\n🎯 Ціль: 750 000 ₴\n\n🔗Посилання на Банку\nhttps://send.monobank.ua/jar/9n75DzdAkM\n\n💳Номер картки Банки\n5375 4112 0181 6381\n\n👉ПРИВАТ\n5168745032031723\nhttps://www.privat24.ua/rd/transfer_to_card/?hash=rd%2Ftransfer_to_card%2F%7B%22from%22%3A%22%22%2C%22to%22%3A%225168745032031723%22%2C%22amt%22%3A%22100%22%2C%22ccy%22%3A%22UAH%22%7D\n\n👉 МОНО\n5375411417173569\n\n👉 PayPal nikitosnata1982@gmail.com\n\nРЕПОСТИМО ДОНАТИМО💪\n\nЗвітую: https://instagram.com/nata_yeromenko\n\nСЛАВА ЗСУ 💙💛\nПИЗДА ₚУСНІ🇺🇦",158377,34,збір коштів,750000,58 бригади\n92 бригади\n750000 ₴\n5375411201816381\n5168745032031723\n5375411417173569\n1982,NaN,NaN
5,1149657308,87344,2022-11-08 14:02:12,Хуевый ✙ Харьков 🇺🇦,Kharkiv,"Друзі, 35 бригаді морської піхоти 18 батальйону все ще потрібна ваша допомога. На зараз зібрано майже 200 тисяч з 270к, давайте піднажмемо та закриємо цей збір.\n\nБанка збору:\nhttps://send.monobank.ua/jar/7pJuFCzb5o\n\nНомер картки: 5375411200387368",139942,0,збір коштів,270000,35 бригаді\n18 батальйону\n200 тисяч\n270к\n5375411200387368,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
278,1390947831,40293,2022-06-19 13:03:05,Харьков 1654 🇺🇦 Город-герой ️,Kharkiv,"⚡️⚡️**Купили бусик! ⚡️⚡️**\nФорд Транзит, 2006 год.\nРебята, вы лучшие!\n\n**Всего задонатили 135 тысяч грн.**\nВыторговали за 134 тысячи грн\nИ на тысячу заправили топливом.\n\n__Во вторник оформляем

In [117]:
df_all_unique[df_all_unique['city'] == 'Lviv'].sort_values('donate')

,channel_id,message_id,date,channel_name,city,message_text,views,forwards,tmp_keyword,donate,tmp_possible_donates,forwarded_from_id,forwarded_from_name
292,1486379119,47872,2022-12-07 08:21:48,✙ Перша приватна мемарня,Lviv,"Тут з Володя Андрєєв розповідає про наш великий збір на 3,6 мільйона гривень. \n\n[Будь ласка, зайдіть та подивіться. \n\n](https://youtu.be/DjusbxWPj04)Від імені Мемарні хочемо подякувати пану Андрію Дрозді ❤️🍻",40401,1,збір коштів,0,"3,6 мільйона",NaN,NaN
522,1486379119,21371,2022-04-09 20:47:29,✙ Перша приватна мемарня,Lviv,"**На глобальній акції зі збору коштів Stand Up for Ukraine в Варшаві вдалося зібрати 9,1 мільярда євро, повідомила Глава Єврокомісії Урсула фон дер Ляйєн. \n\n**Щоб ви собі розуміли, бюджет міста Львів на рік — теж майже 10 мільярдів. Але гривень. \n\nЯкщо на хлопский розум, то за місяць війни зібрали суму, за яку Львів міг би жити плюс-мінус 30 років. \n\n@privatnamemarnya",24311,136,сума збору,0,"9,1 мільярда\n10 мільярдів\n30 років",NaN,NaN
529,1486379119,45282,2022-11-11 14:16:01,✙ Перша приватна мемарня,Lviv,"Неймовірний день. \n\nТа боротьба триває. Роботи ще купа.\n\nНагадуємо, що в нас є актуальний збір 4 тачки для ЗСУ. \n\n💰 **Посилання на МоноБанку\n**https://send.monobank.ua/jar/3Ku3mJX36M\n\n💳 **Картка МоноБанки** - 5375411200475932\n\n💵 **PayPal** - sydary90@gmail.com\n\n@privatnamemarnya",42334,0,монобанка,0,5375411200475932,NaN,NaN
532,1486379119,45071,2022-11-10 11:18:51,✙ Перша приватна мемарня,Lviv,"**У нас є 10 днів, щоб закрити збір. Але попробуємо за тиждень. \n\n**Бо всі ми знаємо, як працюють збори: закрив один, треба починати другий. \n\n💰 **Посилання на МоноБанку\n**https://send.monobank.ua/jar/3Ku3mJX36M\n\n💳 **Картка МоноБанки** - 5375411200475932\n\n💵 **PayPal** - sydary90@gmail.com\n\n👉 Для тих хто може і хоче допомогти готівкою, можна звернутись за номером +380638317071, Максим.",36508,0,монобанка,0,"10 днів\n5375411200475932\n380638317071, Максим",NaN,NaN
538,1486379119,43906,2022-10-26 19:10:42,✙ Перша приватна мемарня,Lviv,"Ілон Маск таки купив твіттер. Ви не Ілони Маски, але своїми донатами робите кориснішу справу, інвестуєте в смерть русні.\n\nКарта: `5375411415345292\n`Монобанка: https://send.monobank.ua/jar/3MyudAV7pd",42931,252,монобанка,0,5375411415345292,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
314,1486379119,31670,2022-06-23 09:29:37,✙ Перша приватна мемарня,Lviv,"Захисникам, що зараз боронять Харків, дуже потрібна наша допомога! \n\nТриває збір коштів на тепловізори для 125 окремої бригади. Залишилось добити **40 000**.\n\nБудь-яка сума справді важлива!\n\nРахунки, на які донатити, можна знайти просканувавши QR-код. У призначені платежів, будь ласка, вказуйте «на тепловізори».\n\nУсі звіти доступні до перегляду на Facebook та Instagram сторінках благодійного фонду «125».\nhttps://www.facebook.com/supportcenter125/\n\nhttps://instagram.com/charitablefoundation125?igshid=YmMyMTA2M2Y=\n\nРазом до перемоги! 💙💛\n\nРахунок:\n```UA733052990000026009011024671\n```Код:\n```44742608\n```Отримувач:\n```БО БФ 125\n\n```Для вашої зручності моно банка:\n\nhttps://send.monobank.ua/jar/YsRL7MXBD\n\n`5375411202408279`",22964,10,збір коштів,44742608,125 окремої\n40000\n125\n733052990000026009011024671\n44742608\n125\n5375411202408279,NaN,NaN
315,1486379119,31360,2022-06-20 14:08:46,✙ Перша приватна мемарня,Lviv,"Захисникам, що зараз боронять Харків, дуже потрібна наша допомога! \n\nТриває збір коштів на тепловізори для 125 окремої бригади. Ціль - 4000€.\n\nБудь-яка сума справді важлива!\n\nРахунки, на які донатити, можна знайти просканувавши QR-код. У призначені платежів, будь ласка, вказуйте «на тепловізори».\n\nУсі звіти доступні до перегляду на Facebook та Instagram сторінках благодійного фонду «125».\nhttps://www.facebook.com/supportcenter125/\n\nhttps://instagram.com/charitablefoundation125?igshid=YmMyMTA2M2Y=\n\nРазом до перемоги! 💙💛\n\nРахунок:\n```UA733052990000026009011024671\n```Код:\n```44742608\n```Отримувач:\n```БО БФ 125\n\